In [11]:
import numpy as np
import sympy

import plotly
import plotly.graph_objects as go
import plotly.express as px

In [116]:


def curvature_discrete_parametric(X: np.array, Y: np.array, Z: np.array):
    # known by MATLAB as surfature
    # where X, Y, Z matrices have a shape (lr+1,lb+1)
    # https://github.com/sujithTSR/surface-curvature/blob/master/surface.py

    (lr, lb) = X.shape

    # First Derivatives
    Xv, Xu = np.gradient(X)
    Yv, Yu = np.gradient(Y)
    Zv, Zu = np.gradient(Z)

    # Second Derivatives
    Xuv, Xuu = np.gradient(Xu)
    Yuv, Yuu = np.gradient(Yu)
    Zuv, Zuu = np.gradient(Zu)

    Xvv, Xuv = np.gradient(Xv)
    Yvv, Yuv = np.gradient(Yv)
    Zvv, Zuv = np.gradient(Zv)

    # Reshape to 1D vectors (same as ravel?)
    Xu = np.reshape(Xu, lr * lb)
    Yu = np.reshape(Yu, lr * lb)
    Zu = np.reshape(Zu, lr * lb)
    Xv = np.reshape(Xv, lr * lb)
    Yv = np.reshape(Yv, lr * lb)
    Zv = np.reshape(Zv, lr * lb)
    Xuu = np.reshape(Xuu, lr * lb)
    Yuu = np.reshape(Yuu, lr * lb)
    Zuu = np.reshape(Zuu, lr * lb)
    Xuv = np.reshape(Xuv, lr * lb)
    Yuv = np.reshape(Yuv, lr * lb)
    Zuv = np.reshape(Zuv, lr * lb)
    Xvv = np.reshape(Xvv, lr * lb)
    Yvv = np.reshape(Yvv, lr * lb)
    Zvv = np.reshape(Zvv, lr * lb)

    Xu = np.c_[Xu, Yu, Zu]
    Xv = np.c_[Xv, Yv, Zv]
    Xuu = np.c_[Xuu, Yuu, Zuu]
    Xuv = np.c_[Xuv, Yuv, Zuv]
    Xvv = np.c_[Xvv, Yvv, Zvv]

    # % First fundamental Coefficients of the surface (E,F,G)
    E = np.einsum("ij,ij->i", Xu, Xu)
    F = np.einsum("ij,ij->i", Xu, Xv)
    G = np.einsum("ij,ij->i", Xv, Xv)

    m = np.cross(Xu, Xv, axisa=1, axisb=1)
    p = np.sqrt(np.einsum("ij,ij->i", m, m))
    n = m / np.c_[p, p, p]

    # % Second fundamental Coefficients of the surface (L,M,N)
    L = np.einsum("ij,ij->i", Xuu, n)
    M = np.einsum("ij,ij->i", Xuv, n)
    N = np.einsum("ij,ij->i", Xvv, n)

    # % Gaussian Curvature
    K = (L * N - M**2) / (E * G - F**2)
    K = np.reshape(K, (lr, lb))

    # % Mean Curvature
    H = (E * N + G * L - 2 * F * M) / (2 * (E * G - F**2))
    H = np.reshape(H, (lr, lb))

    # L = np.reshape(L, lr * lb)
    # M = np.reshape(M, lr * lb)
    # N = np.reshape(N, lr * lb)

    # resape to 1D vectors (lr * lb,)
    L = np.ravel(L)
    M = np.ravel(M)
    N = np.ravel(N)
    
    # % Shape Operator as 3D a matrix of 2D matrices (2, 2, lr*lb)
    LMMN = np.array([[L, M], [M, N]])
    EFFG = np.array([[E, F], [F, G]])
    # reshape so that the 2D matrices are in the last dimension (lr*lb, 2, 2) 
    LMMN = np.swapaxes(LMMN,0,2)
    EFFG = np.swapaxes(EFFG,0,2)

    P = LMMN * np.linalg.inv(EFFG)
    X = np.linalg.eig(P)

    # the result of eig is a tuple of (eigenvalues, eigenvectors)
    k1 = X[0][:, 0] # all the first eigenvalues
    k2 = X[0][:, 1] # all the second eigenvalues
    X1 = X[1][:, 0, :] # all the first eigenvectors
    X1 = X[1][:, 1, :] # all the second eigenvectors

    X1 = np.reshape(X1, (lr, lb))
    X1 = np.reshape(X1, (lr, lb))


    print('k1.shape', k1.shape)
    print('k1', k1)
    print('k2.shape', k2.shape)
    print('k2', k2)
    print('k1vec.shape', X1.shape)
    print('k1vec', X1)
    print('k2vec.shape', X1.shape)
    print('k2vec', X1)
    


    # #% Principal Curvatures
    # k1 = H + np.sqrt(H**2 - K)
    # k2 = H - np.sqrt(H**2 - K)

    return K, H, k1, k2  # , k1vec, k2vec


# TODO: DiffGeoOps off mesh


In [5]:
u, v = sympy.symbols("u v")
f_explicit = sympy.sqrt(1 - v**2)

x, y = u, v

# coordinate range
xx = np.linspace(-1, 1, 20)
yy = np.linspace(-1, 1, 20)

# make coordinate point
X, Y = np.meshgrid(xx, yy)

# dependent variable point on coordinate
f2 = sympy.lambdify((x, y), f_explicit)
Z = f2(X, Y)

fig = go.Figure(go.Surface(x=X, y=Y, z=Z))
fig.show()

In [117]:
K, H, k1, k2 = curvature_discrete_parametric(X, Y, Z)

# print(k2)

fig = px.imshow(k2, labels=dict(x="X", y="Y", color="k2"),)
# fig = go.Figure(go.Surface(x=X, y=Y, z=H))
fig.show()

LMMN.shape (2, 2, 400)
LMMN.shape (400, 2, 2)
P.shape (400, 2, 2)
k1.shape (400,)
k1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

ValueError: px.imshow only accepts 2D single-channel, RGB or RGBA images. An image of shape (400,) was provided. Alternatively, 3- or 4-D single or multichannel datasets can be visualized using the `facet_col` or/and `animation_frame` arguments.